In [1]:
import numpy as np
import pandas as pd
import glob
import os
import time
from tqdm import tqdm
# import matplotlib.pyplot as plt

# Step 3

### Explore merged_tracks_tracksMeta

In [2]:
folder_path = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\02_fps_degraded_data'
file_name_merged_tracks_tracksMeta_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
file_name_merged_tracks_tracksMeta_list

['18_fps_degraded_data.csv',
 '19_fps_degraded_data.csv',
 '20_fps_degraded_data.csv',
 '21_fps_degraded_data.csv',
 '22_fps_degraded_data.csv',
 '23_fps_degraded_data.csv',
 '24_fps_degraded_data.csv',
 '25_fps_degraded_data.csv',
 '26_fps_degraded_data.csv',
 '27_fps_degraded_data.csv',
 '28_fps_degraded_data.csv',
 '29_fps_degraded_data.csv']

In [3]:
# print dataframes info and statastics
def print_df_stats(file_name_merged_tracks_tracksMeta_list, folder_path):
    # go to merged data folder
    os.chdir(folder_path)
    
    for i in range(len(file_name_merged_tracks_tracksMeta_list)):
        print('-------- ',file_name_merged_tracks_tracksMeta_list[i], ":",' --------','\n')
        df=pd.read_csv(file_name_merged_tracks_tracksMeta_list[i])
        # print info
        print(df.info())
        print()
        
        # count how much tracks for each class
        print('count classes:')
        # print(df['class'].value_counts(), '\n')
        unique_trackId_counts = df.groupby('class')['trackId'].nunique()
        print(unique_trackId_counts, '\n')
    
    # change back to parent directory
    os.chdir('..')
 
print_df_stats(file_name_merged_tracks_tracksMeta_list, folder_path)

--------  18_fps_degraded_data.csv :  -------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125216 entries, 0 to 125215
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   recordingId    125216 non-null  int64  
 1   trackId        125216 non-null  int64  
 2   frame          125216 non-null  int64  
 3   trackLifetime  125216 non-null  int64  
 4   xCenter        125216 non-null  float64
 5   yCenter        125216 non-null  float64
 6   heading        125216 non-null  float64
 7   width          125216 non-null  float64
 8   length         125216 non-null  float64
 9   xVelocity      125216 non-null  float64
 10  yVelocity      125216 non-null  float64
 11  class          125216 non-null  object 
dtypes: float64(7), int64(4), object(1)
memory usage: 11.5+ MB
None

count classes:
class
bicycle        92
car           181
pedestrian    154
Name: trackId, dtype: int64 

--------  19_fps_degraded_data.csv :

In [4]:
os.getcwd() # show current directory

'C:\\Users\\yftac\\Documents\\00_Project_Afeka\\ttc_calculation'

## Load merged data to df

In [5]:
# print dataframes info and statastics
def load_merged_dfs(file_name_merged_tracks_tracksMeta_list, folder_path):
    # go to merged data folder
    # folder_name='merged_tracks_tracksMeta_data'
    # change to working dir
    os.chdir(folder_path)
    dfs = []
    for i in range(len(file_name_merged_tracks_tracksMeta_list)):
        print(i)
        df=pd.read_csv(file_name_merged_tracks_tracksMeta_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    print('---- dfs ready ----')
    return dfs

dfs=load_merged_dfs(file_name_merged_tracks_tracksMeta_list, folder_path)

0
1
2
3
4
5
6
7
8
9
10
11
---- dfs ready ----


## Remove parking cars

All parking cars has a 0 velocity and therefore do not contribute to the TTC so we remove them.

In [6]:
def remove_parking_cars(dfs):
    dfs_filtered=[]
    for i in range(len(dfs)):
        df_temp = dfs[i]
        dfs_filtered.append(df_temp.query('xVelocity != 0 and yVelocity != 0'))
    return dfs_filtered

# df_temp_filtered = df_temp.query('xVelocity != 0 and yVelocity != 0')
dfs_filtered = remove_parking_cars(dfs)

In [7]:
len(dfs_filtered)

12

In [8]:
dfs_filtered[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42378 entries, 0 to 125215
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recordingId    42378 non-null  int64  
 1   trackId        42378 non-null  int64  
 2   frame          42378 non-null  int64  
 3   trackLifetime  42378 non-null  int64  
 4   xCenter        42378 non-null  float64
 5   yCenter        42378 non-null  float64
 6   heading        42378 non-null  float64
 7   width          42378 non-null  float64
 8   length         42378 non-null  float64
 9   xVelocity      42378 non-null  float64
 10  yVelocity      42378 non-null  float64
 11  class          42378 non-null  object 
dtypes: float64(7), int64(4), object(1)
memory usage: 4.2+ MB


In [9]:
dfs_filtered[0].describe()

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,yVelocity
count,42378.0,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000
mean,18.0,218.504814,2863.309311,75.506607,50.685412,-28.125652,149.503472,0.462549,1.120974,0.570245,0.037509
std,0.0,118.305121,1650.307323,68.167832,18.603833,9.027841,116.054263,0.793490,1.937267,3.294127,1.238179
min,18.0,0.000000,0.000000,0.000000,9.477080,-55.465010,0.009020,0.000000,0.000000,-13.467670,-7.910850
25%,18.0,116.000000,1421.000000,26.000000,37.232608,-35.457833,14.392630,0.000000,0.000000,-1.230085,-0.677060
50%,18.0,215.000000,2874.000000,54.000000,50.917905,-28.725530,180.991880,0.000000,0.000000,0.531250,0.001280
75%,18.0,320.000000,4244.000000,108.000000,63.231870,-21.054095,208.778328,1.629350,3.573340,1.588723,0.688602
max,18.0,426.000000,5598.000000,444.000000,94.926690,-10.040360,359.999540,2.175130,7.151170,12.582550,9.939700


## Add dimensions to pedestrians and bycicles

in inD dataset pedestrians and bycicle has a width/length of 0. for the ttc calculation dimensions are requiered so we take the following:

In [10]:
dict={'pedestrian_width':     0.5, 
      'pedestrian_length':    0.5, 
      'bycicle_width':        0.8, 
      'bycicle_length':       0.8, 
      }

In [11]:
def add_dim_pedestrian_bicycle(dfs_filtered, dict):
    for i in range(len(dfs_filtered)):
        df_temp = dfs_filtered[i]
        # for each row check class
        for index, row in df_temp.iterrows():
            if row['class'] == 'pedestrian':
                df_temp.at[index, 'width']  = dict['pedestrian_width']
                df_temp.at[index, 'length'] = dict['pedestrian_length']
            elif row['class'] == 'bicycle':
                df_temp.at[index, 'width'] = dict['bycicle_width']
                df_temp.at[index, 'length'] = dict['bycicle_length']
        print(i)
    return dfs_filtered

dfs_filtered=add_dim_pedestrian_bicycle(dfs_filtered, dict)

0
1
2
3
4
5
6
7
8
9
10
11


In [12]:
dfs_filtered[0].describe()

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,yVelocity
count,42378.0,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000,42378.000000
mean,18.0,218.504814,2863.309311,75.506607,50.685412,-28.125652,149.503472,0.889431,1.547855,0.570245,0.037509
std,0.0,118.305121,1650.307323,68.167832,18.603833,9.027841,116.054263,0.556014,1.694219,3.294127,1.238179
min,18.0,0.000000,0.000000,0.000000,9.477080,-55.465010,0.009020,0.500000,0.500000,-13.467670,-7.910850
25%,18.0,116.000000,1421.000000,26.000000,37.232608,-35.457833,14.392630,0.500000,0.500000,-1.230085,-0.677060
50%,18.0,215.000000,2874.000000,54.000000,50.917905,-28.725530,180.991880,0.500000,0.500000,0.531250,0.001280
75%,18.0,320.000000,4244.000000,108.000000,63.231870,-21.054095,208.778328,1.629350,3.573340,1.588723,0.688602
max,18.0,426.000000,5598.000000,444.000000,94.926690,-10.040360,359.999540,2.175130,7.151170,12.582550,9.939700


we can see that width and length have average has grown

In [13]:
# make sure we in the 'root' directory
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"

In [14]:
####### get recordingId_list

# load metadata files
all_files = glob.glob(r'C:\Users\yftac\Documents\00_Project_Afeka\\drone-dataset-tools-master\data\*.csv')
recordingMeta_files_list = [f for f in all_files if 'recordingMeta' in f]
# open recordingMeta files and sort by location and then extract recordingId
def check_string_in_files(files, column_name, search_string):
    matching_files = []
    for file in files:
        df = pd.read_csv(file)
        if search_string in df[column_name].values:
            matching_files.append(file)
    return matching_files
chosen_location=2
# sort by location
chosen_location_files_list_recordingMeta=check_string_in_files(recordingMeta_files_list, 'locationId', chosen_location)
chosen_location_files_list_recordingMeta
recordingId_list=[]
file_name_tracks_list=[]
file_name_tracksMeta_list=[]

# load the names and change to the tracks and tracksMeta file names to cretae file lists of all csv files
for file_path in chosen_location_files_list_recordingMeta:
    # replace name with tracks and tracksMeta
    file_name_tracks=file_path.replace('recordingMeta','tracks')
    file_name_tracksMeta=file_path.replace('recordingMeta','tracksMeta')
    # appen to lists
    file_name_tracks_list.append(file_name_tracks)
    file_name_tracksMeta_list.append(file_name_tracksMeta)
    # create a recordingId list for the "chosen_location"
    file_name='df_'+ os.path.basename(file_name_tracks)
    recordingId=file_name.split("_")[1]
    recordingId_list.append(recordingId)

print("recordingId_list:", recordingId_list, "\n")
# print("tracks files: \n",file_name_tracks_list)
# print("tracksMeta files: \n",file_name_tracksMeta_list)
# print("tracks:\n", "\n".join(map( str, file_name_tracks_list)), "\n")
# print("tracksMeta:\n", "\n".join(map(str, file_name_tracksMeta_list)))

recordingId_list: ['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'] 



In [15]:
### save to csv files  ###
os.chdir(root_dir)

def save_dfs_filtered(dfs_filtered, recordingId_list):
    # create folder to save files 
    folder_name='03_filtered_with_dim_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # change to working dir
    os.chdir(folder_name)
    # initialize a list of the files to return by the function
    file_name_filtered_with_dim_data_list=[]
    # iterate dfs for each df and save
    for x in tqdm(range(len(dfs_filtered)), desc="Saving CSV files"):
        # save as CSV file
        file_name=str(recordingId_list[x]) + '_filtered_with_dim_data' + '.csv'
        dfs_filtered[x].to_csv(file_name, index=False)
        # append the file name to the list
        file_name_filtered_with_dim_data_list.append(file_name)
        
    current_dir = os.getcwd()
    print(f"files saved to: {current_dir}")
    # change back to parent directory
    os.chdir('..')

    return file_name_filtered_with_dim_data_list

file_name_filtered_with_dim_data_list=save_dfs_filtered(dfs_filtered, recordingId_list)

Saving CSV files: 100%|██████████| 12/12 [00:03<00:00,  3.80it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\03_filtered_with_dim_data


In [16]:
file_name_filtered_with_dim_data_list

['18_filtered_with_dim_data.csv',
 '19_filtered_with_dim_data.csv',
 '20_filtered_with_dim_data.csv',
 '21_filtered_with_dim_data.csv',
 '22_filtered_with_dim_data.csv',
 '23_filtered_with_dim_data.csv',
 '24_filtered_with_dim_data.csv',
 '25_filtered_with_dim_data.csv',
 '26_filtered_with_dim_data.csv',
 '27_filtered_with_dim_data.csv',
 '28_filtered_with_dim_data.csv',
 '29_filtered_with_dim_data.csv']

## Load file_name_filtered_with_dim_data_list

In [17]:
# print dataframes info and statastics
def load_dfs(file_name_filtered_with_dim_data_list, root_dir):
    os.chdir(root_dir)
    folder_name='03_filtered_with_dim_data'
    # change to working dir
    os.chdir(folder_name)
    
    dfs = []
    for i in range(len(file_name_filtered_with_dim_data_list)):
        df=pd.read_csv(file_name_filtered_with_dim_data_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    return dfs

dfs_filtered=load_dfs(file_name_filtered_with_dim_data_list, root_dir)